In [7]:
from datascience import *
from pandas import read_stata
import numpy as np

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


In [8]:
def minimize_multi(f, start=None, **vargs):
  def expanded_f(*args):
    return f(args)
  return minimize(expanded_f, start=start, method="L-BFGS-B", **vargs)

In [9]:
data = Table.read_table('Young Lives R1 Ethiopia.csv')
data.show(5)

XXX_ADMIN_VARS_XXX,country,Country,country_code,childid,XXX_OUTCOMES_XXX,zhfa_pr1,zhfa_r1,chheight_r1,XXX_GRADIENTS_XXX,wi_r1,XXX_CHILD_VARS_XXX,female,agechild_r1,ethnicity,bornfirst,lngbfed,bfed_6mo,XXX_MOM_VARS_XXX,edu_cat_dad,edu_cat_mum,agemum_r1,XXX_WEALTH_INDEX_VARS_XXX,hhsize_r1,numroom_r1,wall_r1,roof_r1,floor_r1,radio_r1,fridge_r1,bike_r1,tv_r1,motor_r1,car_r1,mobphone_r1,phone_r1
nan,ET,Ethiopia,Ethiopia,ET030093,nan,-2.89141,-1.79,65,nan,0.305556,nan,Male,6,gurage,0,6,1,nan,Lower primary Complete,Incomplete Upper Primary,30,nan,5,2,adobe/mud,galvanised iron,earth,no,no,no,no,no,no,no,no
nan,ET,Ethiopia,Ethiopia,ET080001,nan,-4.46761,-3.19,60.9,nan,0.0925926,nan,Female,7,other,0,7,1,nan,nan,No Schooling,40,nan,6,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
nan,ET,Ethiopia,Ethiopia,ET120039,nan,nan,nan,nan,nan,0.00617284,nan,Female,6,gurage,0,6,1,nan,No Schooling,No Schooling,40,nan,9,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
nan,ET,Ethiopia,Ethiopia,ET160030,nan,-3.14141,-2.04,64.4,nan,0.0138889,nan,Male,6,hadiva,0,6,1,nan,Lower primary Complete,Lower primary Complete,28,nan,4,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
nan,ET,Ethiopia,Ethiopia,ET170061,nan,1.78859,2.89,74,nan,0.0912698,nan,Male,6,tigrian,0,6,1,nan,Upper Secondary Complete,No Schooling,32,nan,7,1,brick/concrete,wood/planks,earth,no,no,no,no,no,no,no,no


In [10]:
# Remove nans from pertinent columns
dataframe = data.to_df()
altered = dataframe.fillna(-99)
data = Table.from_df(altered)
data.show(5)

XXX_ADMIN_VARS_XXX,country,Country,country_code,childid,XXX_OUTCOMES_XXX,zhfa_pr1,zhfa_r1,chheight_r1,XXX_GRADIENTS_XXX,wi_r1,XXX_CHILD_VARS_XXX,female,agechild_r1,ethnicity,bornfirst,lngbfed,bfed_6mo,XXX_MOM_VARS_XXX,edu_cat_dad,edu_cat_mum,agemum_r1,XXX_WEALTH_INDEX_VARS_XXX,hhsize_r1,numroom_r1,wall_r1,roof_r1,floor_r1,radio_r1,fridge_r1,bike_r1,tv_r1,motor_r1,car_r1,mobphone_r1,phone_r1
-99,ET,Ethiopia,Ethiopia,ET030093,-99,-2.89141,-1.79,65,-99,0.305556,-99,Male,6,gurage,0,6,1,-99,Lower primary Complete,Incomplete Upper Primary,30,-99,5,2,adobe/mud,galvanised iron,earth,no,no,no,no,no,no,no,no
-99,ET,Ethiopia,Ethiopia,ET080001,-99,-4.46761,-3.19,60.9,-99,0.0925926,-99,Female,7,other,0,7,1,-99,nan,No Schooling,40,-99,6,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
-99,ET,Ethiopia,Ethiopia,ET120039,-99,-99,-99,-99,-99,0.00617284,-99,Female,6,gurage,0,6,1,-99,No Schooling,No Schooling,40,-99,9,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
-99,ET,Ethiopia,Ethiopia,ET160030,-99,-3.14141,-2.04,64.4,-99,0.0138889,-99,Male,6,hadiva,0,6,1,-99,Lower primary Complete,Lower primary Complete,28,-99,4,1,wood/branches,straw/thatch,earth,no,no,no,no,no,no,no,no
-99,ET,Ethiopia,Ethiopia,ET170061,-99,1.78859,2.89,74,-99,0.0912698,-99,Male,6,tigrian,0,6,1,-99,Upper Secondary Complete,No Schooling,32,-99,7,1,brick/concrete,wood/planks,earth,no,no,no,no,no,no,no,no


## Multiple Regression

<font color="Blue"> Another way to predict outcomes is with multivariate regression.  Prepare two multivariate regressions to compare: one regression will have all the variables you think are important and the other will have variables you think are not important.  Compare the R^2 of these regressions.  Were you right? Explain.

We will look at the steps to building multivariate regression examining how age and sex predict height.

In [11]:
# Make sure all your data is in integer format and keep only observations with height, age & sex data
data["chheight_r1"]=data.apply(int, '...')
data["agechild_r1"]=data.apply(int, '...')
data["agemum_r1"]=data.apply(int, '...')
# Make a column that will be associated with the intercept
data["For Intercept"]=1

# Select all the variables you want to analyze including intercept and outcome.
# (Until we have the correct minimizing function, just include one variable and the outcome.)
Regression1=data.select("...","...","...","For Intercept")



In [12]:
#remove missing
Regression1=Regression1.where(Regression1['...']>10)
Regression1=Regression1.where(Regression1['...']>10)
Regression1=Regression1.where(Regression1['...']>10)
Regression1

chheight_r1,agechild_r1,agemum_r1,For Intercept
71,11,28,1
72,11,28,1
73,11,35,1
74,12,30,1
56,11,40,1
69,11,20,1
68,11,28,1
68,11,27,1
74,12,22,1
66,11,33,1


In [13]:
# Divide the table in two: predictor variables (sex & age) & outcome variable (height)
X_features_table =Regression1.drop("chheight_r1")
X_true_values = Regression1.column("chheight_r1") # note: this is an array
X_features_table

agechild_r1,agemum_r1,For Intercept
11,28,1
11,28,1
11,35,1
12,30,1
11,40,1
11,20,1
11,28,1
11,27,1
12,22,1
11,33,1


In [14]:
# Choose some intial values for maximizing the coefficients 
# An easy choice is all 0's.  
# There will be the same number of coefficients as there are predictors - a slope for each predictor.
X_initial_coefficient_guess = np.zeros(X_features_table.num_columns)
X_initial_coefficient_guess

array([ 0.,  0.,  0.])

In [15]:
def predict_all(features_table, coefficients):
    """
    Given a table of features called features_table and some coefficients,
    produces linear predictions for each row of features_table.
    
    features_table should be a table with one column for each feature
    being used to predict.  Each row represents a house in the task
    we're doing in this lab.
    
    coefficients should be an array with one element for each column in
    features_table, like the coefficients computed by the function
    least_squares_coefficients.
    
    Returns an array of predictions, one for each row of features_table.
    
    If there were 3 rows in features_table, we would return a 3-element
    array instead, containing the predicted prices for each row.
    """
   # assert features_table.num_columns == len(coefficients), /
    """
    The first argument to predict_all should be a table with one
    column for each feature.  That means it should have the same
    number of columns as the coefficients array (the second
    argument) has elements.
    """
    def predict(features):
        # Given an array of features, produce one prediction.
        return sum(features * coefficients)
    predictions = Table().with_column('features', features_table.rows).apply(predict, 'features')
    return predictions

In [16]:
X_predictions=predict_all(X_features_table, X_initial_coefficient_guess)
X_predictions

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [17]:
# change the initial coefficient guess up above to make a different prediction!
X_new_coefficient_guess = [1]
X_predictions=predict_all(X_features_table, X_new_coefficient_guess)
X_predictions

array([40, 40, 47, ..., 56, 44, 51])

In [18]:
def compute_errors(features_table, coefficients, true_values):
    """
    Computes the prediction errors for a linear model with the given
    coefficients when predicting the true values for the given
    examples.
    
    features_table should be a table with one column for each feature
    being used to predict.  
    
    coefficients should be an array of numbers, one for each feature.
    
    true_values should be an array of numbers, one for each row in
    features_table.  It records the true prices of each house.
    """
    return predict_all(features_table, coefficients) - true_values



In [19]:
X_errors=compute_errors(X_features_table, X_initial_coefficient_guess, X_true_values)
X_errors

array([-71., -72., -73., ..., -74., -64., -75.])

In [20]:
def rmse(errors):
    """
    Computes the root mean squared error when a regression model makes
    the given errors.  So errors should be an array of numbers, one for
    each row in some data table for which we're computing predictions.  
    Each number is the prediction error of some regression model.
    """
    return np.mean(errors**2)**0.5



In [21]:
r2=rmse(X_errors)
r2

72.658466978264997

In [22]:
def make_least_squares_objective_function(features_table, true_values):
    """
    Makes an objective function for main data in the features_table
    table, where the true values we're trying to predict are true_values.
    
    features_table should be a table with one column for each feature
    being used to predict.  
    
    true_values should be an array of numbers, one for each row in
    features_table.  
    
    The returned value is a function.  That function takes an array of
    coefficients and returns a number.  Larger values of that number
    mean that those coefficients produce worse prediction errors.
    """
    def objective_function(coefficients):
        errors = compute_errors(features_table, np.array(coefficients), true_values)
        return rmse(errors)
    return objective_function



In [23]:
X_objective_function = make_least_squares_objective_function(X_features_table, X_true_values)
X_objective_function

<function __main__.make_least_squares_objective_function.<locals>.objective_function>

In [24]:
def least_squares_coefficients(main_data, predicted_column_name):
    """
    Performs multiple linear regression predicting predicted_column_name
    using the other columns of main_data as features.
    
    main_data should be a table with one column for each feature
    being used to predict, plus one column for the value we're trying
    to predict.  That column's name should equal predicted_column_name.
    Each row represents a house in the task we're doing in this lab.
    
    predicted_column_name should be a string, the name of the column in
    main_data that we're trying to predict.
    
    Returns an array of coefficients, one for each feature (that is, one
    for each column in main_data other than predicted_column_name).
    
    For example, if main_data has 3 columns, mom's education, age, and height,
    and predicted_column_name is "Price", then we will use mom's education and
    age to predict height.  This function will return an array of 2
    numbers, a regression coefficient for mom's education and a
    regression coefficient for age.
    """
    features_table = main_data.drop(predicted_column_name)
    true_values = main_data.column(predicted_column_name)
    objective_function_OLS = make_least_squares_objective_function(features_table, true_values)
    
    # Now we find the coefficients that produce the smallest
    # error.
    initial_coefficient_guess = np.zeros(features_table.num_columns)
    best_coefficients = minimize_multi(objective_function_OLS, start=initial_coefficient_guess)
    if features_table.num_columns == 1:
        return np.array([best_coefficients])
    else:
        return best_coefficients

In [25]:
X_best_coefficients = minimize_multi(X_objective_function, start=X_initial_coefficient_guess)
X_best_coefficients

array([  8.30829197e-01,   9.87849923e-03,   6.05016418e+01])

In [26]:
My_Coefficients = least_squares_coefficients(Regression1, 'chheight_r1')
My_Coefficients

array([  8.30829197e-01,   9.87849923e-03,   6.05016418e+01])

Let's see how well our model fits the data.  We will calculate the R^2.

In [27]:
mean_height=np.average(Regression1["..."])
mean_height

72.491746307558643

In [28]:
My_predictions=predict_all(X_features_table, My_Coefficients)
Regression1["Predicted"]=My_predictions
Regression1["Diff_Predict_SQ"]=(Regression1["..."]-Regression1["Predicted"])**2
Regression1["Diff_True_SQ"]=(Regression1["..."]-mean_height)**2
Regression1

chheight_r1,agechild_r1,agemum_r1,For Intercept,Predicted,Diff_Predict_SQ,Diff_True_SQ
71,11,28,1,69.9174,1.17211,2.22531
72,11,28,1,69.9174,4.33739,0.241814
73,11,35,1,69.9865,9.08112,0.258322
74,12,30,1,70.7679,10.4462,2.27483
56,11,40,1,70.0359,197.007,271.978
69,11,20,1,69.8383,0.702802,12.1923
68,11,28,1,69.9174,3.67627,20.1758
68,11,27,1,69.9075,3.63849,20.1758
74,12,22,1,70.6889,10.9633,2.27483
66,11,33,1,69.9668,15.7351,42.1428


In [29]:
r_squared=1-np.sum(Regression1["Diff_Predict_SQ"])/np.sum(Regression1["Diff_True_SQ"])
r_squared

0.12176060176693848

Let's be more systematic about building the model - we will add variables to try to get the largest R^2.  Adding more covariates will improve the fit of the model, but we are also making the model more complex.  Let's add covariates in order of importance, keeping those that increase the adjusted R^2.  The formula for the adjusted R^2 is 1-(1-R^2)(N-1)/(N-p-1), where N= sample size and p= number of predictors.

This model is of correlation, not causation.  But since this is a first indication of a potential causal relationship, we may as well try a policy and then later see if it works out. <font color="Blue"> If you were attempting to change the outcome based on changing the feature, which feature would you try to change?  Don't just consider the one with the highest correlation, but also take into account costs and difficulty of changing the feature. (Not a data exercise)